In [1]:
!pip install concrete-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of onnxconverter-common to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.6/626.6 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.0/151.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.1/678.1 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [5]:
# model_train.py
from concrete.ml.sklearn import LogisticRegression
import numpy as np
import time
import pickle

# 模擬資料（UID hash 對應風險等級）
X_train = np.random.randint(0, 2**16, size=(300, 1)).astype(np.float32)
y_train = X_train[:, 0].astype(int) % 3  # 三分類：低中高風險（0/1/2）

# 建立與編譯模型
model = LogisticRegression()
model.fit(X_train, y_train)

print("⏳ 編譯 FHE 模型中...")
start_time = time.time()
model.compile(X_train)
print(f"✅ 編譯完成，耗時 {time.time() - start_time:.2f} 秒")

# 儲存模型（會包含 FHE artifact）
#model.save("fhe_model")
#print("✅ FHE 模型已儲存")

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

# some time later...

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)


⏳ 編譯 FHE 模型中...
✅ 編譯完成，耗時 0.26 秒


ValueError: ctypes objects containing pointers cannot be pickled

In [9]:
from concrete import fhe
import numpy as np

# 定義：比較兩個 UID 是否一致
@fhe.compiler({"x": "encrypted", "y": "encrypted"})
def is_equal(x, y):
    return x == y

# 提供 inputset（樣本輸入）給編譯器
example_input = (np.array([12345], dtype=np.uint16), np.array([54321], dtype=np.uint16))
compiled_circuit = is_equal.compile(inputset=[example_input])

# 模擬兩家機構的 UID hash（相同）
uid_hash_a = np.array([12345], dtype=np.uint16)
uid_hash_b = np.array([12435], dtype=np.uint16)

# 一起加密為一組 (x, y)
enc_inputs = compiled_circuit.encrypt(uid_hash_a, uid_hash_b)

# 執行與解密
enc_result = compiled_circuit.run(enc_inputs)
result = compiled_circuit.decrypt(enc_result)

print("✅ 是否為相同 UID：", bool(result[0]))

# 沒有keygen也能執行？

✅ 是否為相同 UID： False


In [13]:
from concrete import fhe
import numpy as np

@fhe.compiler({"x": "encrypted", "y": "encrypted"})
def is_equal(x, y):
    return x == y

example_input = (np.array([12345], dtype=np.uint16), np.array([54321], dtype=np.uint16))
compiled_circuit = is_equal.compile(inputset=[example_input])

# 明文 UID
plaintext = np.array([12345], dtype=np.uint16)

# 加密兩次同一個 UID
cipher1 = compiled_circuit.encrypt(plaintext, plaintext)
cipher2 = compiled_circuit.encrypt(plaintext, plaintext)

# 執行比較
enc_result = compiled_circuit.run(cipher1)
result = compiled_circuit.decrypt(enc_result)

# 顯示結果與密文摘要
print("✅ 是否為相同 UID：", bool(result[0]))
print("🔒 Cipher1 hash:", hash(str(cipher1)))
print("🔒 Cipher2 hash:", hash(str(cipher2)))
print("🔍 Cipher1 == Cipher2 ?", cipher1 == cipher2)


✅ 是否為相同 UID： True
🔒 Cipher1 hash: -2050800770822809652
🔒 Cipher2 hash: -4818875346215178825
🔍 Cipher1 == Cipher2 ? False


In [17]:
# FHE 白名單驗證
from concrete import fhe
import numpy as np

# 假設我們允許的 UID 是以下這些（事先定義好的白名單）
allowlist = [1111, 2222, 3333, 4444, 5555]

# 建立允許名單比對邏輯：如果 uid == 任一 allowlist[i] → return True
def uid_in_allowlist(uid):
    return sum([uid == np.uint16(x) for x in allowlist]) > 0

# 編譯器：只有一個密文輸入 uid
compiler = fhe.Compiler(uid_in_allowlist, {"uid": "encrypted"})

# Inputset：提供一些樣本 UID 給編譯器估參數
inputset = [(np.uint16(x),) for x in [1111, 9999, 2222, 8888, 3333]]

print("🚧 編譯中...")
circuit = compiler.compile(inputset)

print("🔐 產生密鑰...")
circuit.keygen()

# 要測試的 UID（是否在 allowlist 中）
# test_uid = np.array([2222], dtype=np.uint16)
test_uid1 = np.uint16(2222)
test_uid2 = np.uint16(9999)

# 加密並執行判斷
enc_uid1 = circuit.encrypt(test_uid1)
enc_result1 = circuit.run(enc_uid1)
result1 = circuit.decrypt(enc_result1)

enc_uid2 = circuit.encrypt(test_uid2)
enc_result2 = circuit.run(enc_uid2)
result2 = circuit.decrypt(enc_result2)

#print(f"✅ 是否為 allowlist UID？ → {bool(result[0])}")
print(f"2222 是否為 allowlist UID？ → {result1}")
print(f"9999 是否為 allowlist UID？ → {result2}")

🚧 編譯中...
🔐 產生密鑰...
2222 是否為 allowlist UID？ → 1
9999 是否為 allowlist UID？ → 0


In [12]:
# Example from https://docs.zama.ai/concrete/get-started/quick_start
from concrete import fhe

def add(x, y):
    return x + y

compiler = fhe.Compiler(add, {"x": "encrypted", "y": "encrypted"})

inputset = [(2, 3), (0, 0), (1, 6), (7, 7), (7, 1), (3, 2), (6, 1), (1, 7), (4, 5), (5, 4)]

print(f"Compilation...")
circuit = compiler.compile(inputset)

print(f"Key generation...")
circuit.keygen()

print(f"Homomorphic evaluation...")
encrypted_x, encrypted_y = circuit.encrypt(2, 6)
encrypted_result = circuit.run(encrypted_x, encrypted_y)
result = circuit.decrypt(encrypted_result)

assert result == add(2, 6)

print(result)

Compilation...
Key generation...
Homomorphic evaluation...
8
